# 初始化

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import panel as pn
pn.extension()

sys.path.append(os.path.abspath('../'))
os.environ["NUMBA_CUDA_DRIVER"] = "C:\\Windows\\System32\\nvcuda.dll"


In [ ]:
from util.data_utils import load_csv_folder

# 加载数据
data_dir = '../../data/sat1'  # 路径可根据实际情况调整
dataframes = load_csv_folder(data_dir)
file = list(dataframes.keys())[0]  # 选择第一个文件演示
df = dataframes[file]
print(f'加载数据文件: {file}, 行数: {len(df)}')
df.head()

# 预处理

In [ ]:
from downsample.downsample import lttb

n=105120
downsampled = {}
for file in dataframes.keys():
    df = dataframes[file]
    downsampled[file] = lttb(df,n)

# 实验

In [54]:
import stumpy
import numpy as np

m=288
L=m
n=4


results = {}
for file in downsampled.keys():
    # 数据加载
    ay=downsampled[file].values.ravel()

    # 计算
    mp=stumpy.stump(ay, m=m)
    cac=stumpy.fluss(mp[:,1],L,n)

    # 结果保存
    length = min(len(ay),len(mp),len(cac[0]))

    result = downsampled[file].iloc[(len(ay)-length):].copy()
    result.rename(columns={file:'raw'},inplace=True)
    result['mp']=mp[len(mp)-length:,0]
    result['mpi']=mp[len(mp)-length:,1]
    result['mpl']=mp[len(mp)-length:,2]
    result['mpr']=mp[len(mp)-length:,3]
    result['cac']=cac[0]
    result['flag']=0
    result.iloc[cac[1],-1]=1
    
    results[file]=result


# 可视化

In [ ]:
import panel as pn
import hvplot.pandas

channel_selector = pn.widgets.Select(name="选择数据")
channel_selector.options = list(results.keys())
channel_selector.value = list(results.keys())[0]

data_selector = pn.widgets.MultiSelect(name="选择数据")
data_selector.options = list(results[channel_selector.value].columns)

def plot_line(dfs, channel, name):
    df = dfs[channel]
    plot = df[name].hvplot.line(
        title=name,
        width=1000,
        height=300,
    ).opts(
        # toolbar='disable',
        backend_opts={
            "x_range.bounds": (
                df.index.min(),
                df.index.max(),
            ),  # optional: limit max viewable x-extent to data
            "y_range.bounds": (
                df[name].min().item() - 1,
                df[name].max().item() + 1,
            ),  # optional: limit max viewable y-extent to data
        }
    )
    return plot


@pn.depends(channel_selector.param.value, data_selector.param.value)
def chart(channel,data):
    if channel is None or data is None:
        return pn.Column()
    charts=[]
    for d in data:
        charts.append(plot_line(results,channel,d))
    return pn.Column(*charts)


panel = pn.Column(pn.Row(channel_selector,data_selector), chart)
panel

# 分析